In [2]:
suppressMessages({
    library(SingleCellExperiment)
    #library(zinbwave)
    library(scRNAseq)
    library(matrixStats)
    library(magrittr)
    library(edgeR)
    library(scran)
})

In [3]:
get_gene_families <- function(families){
    base_name = '/home/soma/Documents/Newest/analysisfiles/%smouse.txt'
    
    all_genes <- c()
    
    for (family in families){
        fname = sprintf(base_name, family)
        x <- scan(fname, what="", sep="\n")
        lines <- strsplit(x, "[[:space:]]+")
        genes <- lapply(lines, `[[`, 1)
        
        all_genes <- c(all_genes, genes)
    }
    
    all_genes <- unique(all_genes)
    
    return (all_genes)
}

get_dataset <- function(dataset){
    fname <- sprintf('Datasets/%s_edgeR.RData', dataset)
    dge <- readRDS(fname)
    genes <- get_gene_families('CAM')
    
    dge <- dge[row.names(dge) %in% genes,]
    
    return (dge)
}

get_cell_labels <- function(dataset){
    fname <- sprintf('Datasets/%s-transcriptional_labels.tsv', dataset)
    cell_labels <- read.table(fname, sep='\t', header=1, row=1)
    
    return (cell_labels)
}

get_weighted_vars <- function(dge, ctype1, ctype2){
    ctypes <- dge$samples$group
    num1 = sum(ctypes %in% ctype1)
    num2 = sum(ctypes %in% ctype2)
    weights = rep(num1, length(ctypes))
    weights[ctypes %in% ctype1] = num2

    wvars <- rowWeightedVars(cpm(dge) %>% log1p, w=weights)
    
    return (wvars)
}

calc_weighted_vars <- function(dge, ctype1, ctype2){
    ctypes <- dge$samples$group
    count_mat <- cpm(dge)
    weights = rep(0, length(ctypes))
    for (ctype in unique(ctypes)){
        weights[ctypes==ctype] = 1 / sum(ctypes==ctype)
    }
    weights <- round(weights / min(weights))
    
    wvars <- rowWeightedVars(count_mat %>% log1p, w=weights)
    
    return (wvars)
}

filter_features <- function(dge, ctype1, ctype2){
    dge <- dge[,dge$samples$group %in% c(ctype1, ctype2)]
    
    keep <- rowSums(cpm(dge)>1)>4
    #keep2 <- aveLogCPM(dge) > 4
    #keep <- keep1 & keep2
    dge <- dge[keep,,keep.lib.sizes=FALSE]
    #cpm(dge) %>% log1p %>% rowVars -> vars
    vars <- get_weighted_vars(dge, ctype1, ctype2)
    names(vars) <- rownames(dge)
    vars <- sort(vars, decreasing = TRUE)
    #dge <- dge[names(vars)[1:5000],]
    
    return (dge)
}

run_comparison <- function(dge, ctype1, ctype2, fname){
    # get data of interest
    dge <- filter_features(dge, ctype1, ctype2)
    data = dge$samples
    data$group <- factor(data$group, levels=c(ctype1, ctype2))
    design <- model.matrix(~group, data = data)
    
    # run analysis
    dge <- estimateDisp(dge, design)
    #fit <- glmFit(dge, design, robust=TRUE)
    fit <- glmQLFit(dge, design, robust=TRUE)
    lrt <- glmQLFTest(fit, coef=2)
    #lrt <- glmWeightedF(fit, coef = 2)
    #lrt <- glmTreat(fit, coef=2, lfc=1)
    
    save_edgeR_result(lrt, fname)
}

run_multi_comparisons <- function(dge, ctypes1, ctypes2, fname){
    # get data of interest
    dge <- filter_features(dge, ctypes1, ctypes2)
    data <- dge$samples
    data$group <- factor(data$group, levels=c(levels(data$group), c('Type_1', 'Type_2')))
    data[data$group %in% ctypes1, 'group'] <- 'Type_1'
    data[data$group %in% ctypes2, 'group'] <- 'Type_2'
    data$group <- factor(data$group, levels=c('Type_1', 'Type_2'))
    design <- model.matrix(~group, data = data)
    
    # run analysis
    dge <- estimateDisp(dge, design)
    fit <- glmQLFit(dge, design, robust=TRUE)
    lrt <- glmQLFTest(fit, coef=2)
    
    save_edgeR_result(lrt, fname)
}

save_edgeR_result <- function(lrt, fname){
    txtname <- sprintf('Differential/edgeR/%s.txt', fname)
    df <- topTags(lrt, n=dim(lrt$table)[1])$table
    df <- df[order(df$PValue), ]
    write.table(df, file=txtname, sep='\t', quote=FALSE)
}

run_all_comparisons <- function(dge, labels, folder){
    dge$samples$group <- labels
    group = as.character(unique(labels))
    
    for (i in 1:(length(group)-1)){
        for (j in (i+1):length(group)){
            ctype1 <- group[[i]]
            ctype2 <- group[[j]]
            title <- sprintf('%s/%s vs %s', folder, ctype1, ctype2)
            run_comparison(dge, ctype1, ctype2, title)
        }
    }
}

In [3]:
dge <- get_dataset('Lab_Pvalb')
cell_labels <- get_cell_labels('Lab_Pvalb')
cell_labels <- cell_labels[colnames(dge),]
labels <- cell_labels$Morph.PV.types
run_all_comparisons(dge, labels, 'Morph_CAM')

In [7]:
dge <- get_dataset('Lab_Dataset')
cell_labels <- dge$samples
labels <- cell_labels$group
run_multi_comparisons(dge, c('SST-OLM'), c('vAAC', 'vBIC', 'hBIC', 'vBC', 'hBC'), 'Lab_CAM/Sst-OLM vs Pvalb')

Warning message in gzfile(file, "rb"):
“cannot open compressed file 'Datasets/Lab_Data_edgeR.RData', probable reason 'No such file or directory'”

ERROR: Error in gzfile(file, "rb"): cannot open the connection
